In [45]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import tqdm
import cv2

In [2]:
# path = '../../data/jpg/global_F814W_InvertY/'
path_sub_train = '../../data/ds9_dataset/labels_sub_txt/train/'
path_sub_test = '../../data/ds9_dataset/labels_sub_txt/test/'
sub_folder_train = glob.glob(os.path.join(path_sub_train, "*"))
sub_folder_test = glob.glob(os.path.join(path_sub_test, "*"))

In [3]:
sub_folder_train

['../../data/ds9_dataset/labels_sub_txt/train\\ASINH',
 '../../data/ds9_dataset/labels_sub_txt/train\\Linear',
 '../../data/ds9_dataset/labels_sub_txt/train\\Log',
 '../../data/ds9_dataset/labels_sub_txt/train\\Power',
 '../../data/ds9_dataset/labels_sub_txt/train\\SINH',
 '../../data/ds9_dataset/labels_sub_txt/train\\Sqrt',
 '../../data/ds9_dataset/labels_sub_txt/train\\Squarded']

In [15]:
# revers origin image coordinate is important, make sure that the origin is at the bottom left corner same as the DS9 coordinate
# ref by https://stackoverflow.com/questions/43584463/change-origin-of-image-coordinate-system-to-bottom-left-instead-of-default-top-l

In [5]:
# reverse the origin of the y-axis of the image
def reverse_origin(image):
    plt.imshow(image, origin='lower')
    plt.show()
    

# display the image
def display_image(image):
    plt.imshow(image)
    plt.show()
# image = plt.imread(files_name[0])
# display_image((image))

In [16]:
# display image in file_
index = 0
img = plt.imread(files_name[index])


In [17]:
# read text in position file
# path_ = "../../data/jpg/labels_global_F814W_InvertY/"
path_ = "../../data/data_augmentation/gray/x_y_coordinate/"
file_ = glob.glob(os.path.join(path_, "*"))
with open(file_[index], "r") as f:
    items_list = f.readlines()
    items_list = [item.strip().split(" ") for item in items_list]


In [18]:
len(items_list), items_list

(0, [])

In [6]:
# plot scatter in image using position file by x_min = x-w, x_max = x+w, y_min = y-h, y_max = y+h
def plot_scatter(items_list, img):
    """
    items_list: list of items in position file
    img: image
    """
    for item in items_list:
        x = float(item[1])
        y = float(item[2])
        w = float(item[3])
        h = float(item[4])
        x_min = x - w
        x_max = x + w
        y_min = y - h
        y_max = y + h
        plt.scatter(x_min, y_min, c='r')
        plt.scatter(x_max, y_max, c='r')
        plt.scatter(x_min, y_max, c='r')
        plt.scatter(x_max, y_min, c='r')
        plt.scatter(x, y, c='b')
    plt.imshow(img)
    plt.show()

# plot_scatter(items_list, img)

In [7]:
# display bounding box by using position file x_min = x-h, x_max = x+h, y_min = y-w, y_max = y+w
def plot_bounding_box(items_list, img):
    """
    items_list: list of items in position file
    img: image
    """
    plt.rcParams['figure.figsize'] = [15, 15]
    for item in items_list:
        x = float(item[1])
        y = float(item[2])
        w = float(item[3])
        h = float(item[4])
        x_min = x - w
        x_max = x + w
        y_min = y - h
        y_max = y + h
        plt.plot([x_min, x_max], [y_min, y_min], c='r')
        plt.plot([x_min, x_max], [y_max, y_max], c='r')
        plt.plot([x_min, x_min], [y_min, y_max], c='r')
        plt.plot([x_max, x_max], [y_min, y_max], c='r')
    plt.imshow(img)
    plt.show()

# plot_bounding_box(items_list, img)

# do it with loop

In [54]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    # return x,y,w,h with float 4 digits
    return (round(x, 4), round(y, 4), round(w, 4), round(h, 4))

In [57]:
# convert the positon from pixel to YOLO format following the format file: labels x_center y_center width height
# calculate x_min = x-w, x_max = x+w, y_min = y-h, y_max = y+h
def convert_position_to_YOLO_format(items_list, img, name_to_write , path):
    """
    items_list: list of items in position file
    img: image
    """
    try:
        height, width, _ = img.shape
    except:
        height, width = img.shape
    if items_list != []:
        for item in items_list:
            x = float(item[1])
            y = float(item[2])
            w = float(item[3])
            h = float(item[4])
            x_min = (x - w) 
            x_max = (x + w) 
            y_min = (y - h) 
            y_max = (y + h) 
            new_pos = convert((width, height), (x_min, x_max, y_min, y_max))
            # open file to write in Yolo_labels_global_F814W_InvertY by file_
            with open(path + name_to_write, "a") as f:
                if item[0] == "Cluster":
                    # print("0", new_pos[0], new_pos[1], new_pos[2], new_pos[3])
                    f.write("0" + " " + str(new_pos[0]) + " " + str(new_pos[1]) + " " + str(new_pos[2]) + " " + str(new_pos[3]) + "\n")
                else:
                    # print("1", new_pos[0], new_pos[1], new_pos[2], new_pos[3])
                    # f.write("1" + " " + str(new_pos[0]) + " " + str(new_pos[1]) + " " + str(new_pos[2]) + " " + str(new_pos[3]) + "\n")
                    
                    # don't write anything if the item is not a cluster
                    pass
            f.close()
    else:
        with open(path + name_to_write, "a") as f:
            f.write("")
        f.close()
                
# convert_position_to_YOLO_format(items_list, img)


In [13]:
# loop for saving all files as a new file in path ../data/jpg/Yolo_labels_global_F814W_InvertY/
for index in range(len(files_name)):
    img_name = files_name[index].split("\\")[-1].split(".")[0]
    txt_name = file_[index].split("\\")[-1].split(".")[0]
    if img_name == txt_name:
        img = plt.imread(files_name[index])
        with open(file_[index], "r") as f:
            items_list = f.readlines()
            items_list = [item.strip().split(" ") for item in items_list]
            print("Processing file: ", txt_name)
            convert_position_to_YOLO_format(items_list, img, txt_name + ".txt", "../../data/data_augmentation/labels/")
    else:
        print("File name is not match, Example: ", img_name, txt_name)

Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_1
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_10
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_11
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_12
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_13
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_14
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_15
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_16
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_17
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_18
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_19
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_2
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_20
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_21
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_22
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_23
Processing file:  hst_10190_12_acs_wfc_f814w_j90o12_drc_24

with filters

In [10]:
def load_obj(path, file_type): # this function returned file path
    import glob
    import os
    if file_type == '-': # for loading subfolder
        return glob.glob(os.path.join(path))
    elif file_type == '*' :# means all
        return glob.glob(os.path.join(path, "*"))
    else : # require file type
        return glob.glob(os.path.join(path, "*.{}".format(file_type)))

In [8]:
fillter_list = ['Linear', 'Log', 'Power', 'Sqrt', 'Squarded', 'ASINH', 'SINH']
path_filters = load_obj("../../data/data_augmentation/24_images/only_obj_cropped/", '*')
path_filters = path_filters[:-1]
path_filters

['../../data/data_augmentation/24_images/only_obj_cropped\\ASINH',
 '../../data/data_augmentation/24_images/only_obj_cropped\\Linear',
 '../../data/data_augmentation/24_images/only_obj_cropped\\Log',
 '../../data/data_augmentation/24_images/only_obj_cropped\\Power',
 '../../data/data_augmentation/24_images/only_obj_cropped\\SINH',
 '../../data/data_augmentation/24_images/only_obj_cropped\\Sqrt',
 '../../data/data_augmentation/24_images/only_obj_cropped\\Squarded']

In [19]:
path_coor = load_obj("../../data/data_augmentation/24_images/coordinate_contain_obj/", '*')
path_coor = path_coor[:-1]
path_coor

['../../data/data_augmentation/24_images/coordinate_contain_obj\\ASINH',
 '../../data/data_augmentation/24_images/coordinate_contain_obj\\Linear',
 '../../data/data_augmentation/24_images/coordinate_contain_obj\\Log',
 '../../data/data_augmentation/24_images/coordinate_contain_obj\\Power',
 '../../data/data_augmentation/24_images/coordinate_contain_obj\\SINH',
 '../../data/data_augmentation/24_images/coordinate_contain_obj\\Sqrt',
 '../../data/data_augmentation/24_images/coordinate_contain_obj\\Squarded']

In [11]:
len(path_filters)

7

In [11]:


fillter_list = ['Linear', 'Log', 'Power', 'Sqrt', 'Squarded', 'ASINH', 'SINH']
path_filters = load_obj("../../data/data_augmentation/24_images/only_obj_cropped/testing_data/", '*')
path_coor = load_obj("../../data/data_augmentation/24_images/coordinate_contain_obj/testing_data/", '*')

In [10]:
# for testing data
# path_filters = load_obj("../../data/data_augmentation/24_images/full_img_cropped/testing_data/", '*')
# path_coor = load_obj("../../data/data_augmentation/24_images/coordinate/testing_data/", '*')

# path_filters = load_obj("../../data/data_augmentation/24_images/only_obj_cropped/testing_data/", '*')
# path_coor = load_obj("../../data/data_augmentation/24_images/coordinate_contain_obj/testing_data/", '*')

In [ ]:
# ไฟล์ coordinate "ไม่ได้" ใช้ร่วมกันทุก filter 

In [12]:
# loop for saving all files as a new file
for index in range(len(fillter_list)):

    temp_fil_name = fillter_list[index]
    for w in path_filters:
        if temp_fil_name in w:
            temp_path = w
            break
    
    for q in path_coor:
        if temp_fil_name in q:
            temp_coor = q
            break

    print('process filters: ', temp_fil_name)

    # print(temp_path, temp_coor)
    sub_files_name = load_obj(temp_path, '*')
    sub_file_coor = load_obj(temp_coor, '*')
    # print(sub_files_name)
    # print(sub_file_coor)

    counter = 0
    for index_ in range(len(sub_files_name)):
        img_name = sub_files_name[index_].split("\\")[-1].split(".")[0]
        txt_name = sub_file_coor[index_].split("\\")[-1].split(".")[0]
        if img_name == txt_name:
            counter += 1
            img = plt.imread(sub_files_name[index_])
            with open(sub_file_coor[index_], "r") as f:
                items_list = f.readlines()
                items_list = [item.strip().split(" ") for item in items_list]
                # print("Processing file: ", txt_name)
                # convert_position_to_YOLO_format(items_list, img, txt_name + ".txt", "../../data/data_augmentation/24_images/labels/" + temp_fil_name + "/")
                convert_position_to_YOLO_format(items_list, img, txt_name + ".txt",
                 "../../data/data_augmentation/24_images/labels/testing_data/" + temp_fil_name + "/")
        else:
            print("File name is not match, Example: ", img_name, txt_name)
    print("Done!")
    print("a number of objects that processed:", counter)
print("\nDone all!")

process filters:  Linear
Done!
a number of objects that processed: 21
process filters:  Log
Done!
a number of objects that processed: 21
process filters:  Power
Done!
a number of objects that processed: 21
process filters:  Sqrt
Done!
a number of objects that processed: 21
process filters:  Squarded
Done!
a number of objects that processed: 21
process filters:  ASINH
Done!
a number of objects that processed: 21
process filters:  SINH
Done!
a number of objects that processed: 21

Done all!


# Procesing ds9 images

In [67]:
sub_folder_train, sub_folder_test

(['../../data/ds9_dataset/labels_sub_txt/train\\ASINH',
  '../../data/ds9_dataset/labels_sub_txt/train\\Linear',
  '../../data/ds9_dataset/labels_sub_txt/train\\Log',
  '../../data/ds9_dataset/labels_sub_txt/train\\Power',
  '../../data/ds9_dataset/labels_sub_txt/train\\SINH',
  '../../data/ds9_dataset/labels_sub_txt/train\\Sqrt',
  '../../data/ds9_dataset/labels_sub_txt/train\\Squarded'],
 ['../../data/ds9_dataset/labels_sub_txt/test\\ASINH',
  '../../data/ds9_dataset/labels_sub_txt/test\\Linear',
  '../../data/ds9_dataset/labels_sub_txt/test\\Log',
  '../../data/ds9_dataset/labels_sub_txt/test\\Power',
  '../../data/ds9_dataset/labels_sub_txt/test\\SINH',
  '../../data/ds9_dataset/labels_sub_txt/test\\Sqrt',
  '../../data/ds9_dataset/labels_sub_txt/test\\Squarded'])

In [28]:
path_sub_img_train = '../../data/ds9_dataset/sub_img/train/*'
path_sub_img_test = '../../data/ds9_dataset/sub_img/test/*'

In [64]:
path_save_train = '../../data/ds9_dataset/labels_yolo/train/'
path_save_test = '../../data/ds9_dataset/labels_yolo/test/'

# For training set

In [61]:
# loop for saving all files as a new file
for index in tqdm.tqdm(range(len(sub_folder_train))):
    fil_name = sub_folder_train[index].split("\\")[-1].split(".")[0]
    print('process filters: ', fil_name)
    sub_iterate_labels = glob.glob(os.path.join(sub_folder_train[index], "*.txt"))
    sub_itearate_img = glob.glob(os.path.join(path_sub_img_train))[index]
    sub_itearate_img = glob.glob(os.path.join(sub_itearate_img, "*.jpg"))
    # print(len(sub_iterate_labels))
    # print(len(sub_itearate_img))

    for index_ in tqdm.tqdm(range(len(sub_itearate_img))):
        img_name = sub_itearate_img[index_].split("\\")[-1].split(".")[0]
        txt_name = sub_iterate_labels[index_].split("\\")[-1].split(".")[0]
        if img_name == txt_name:
            # read image with cv2
            img = cv2.imread(sub_itearate_img[index_], 0)
            with open(sub_iterate_labels[index_], "r") as f:
                items_list = f.readlines()
                items_list = [item.strip().split(" ") for item in items_list]
                # print("Processing file: ", txt_name)
                convert_position_to_YOLO_format(items_list, img, txt_name + ".txt",
                    path_save_train + fil_name + "/")
        else:
            print("File name is not match, Example: ", img_name, txt_name)

  0%|          | 0/7 [00:00<?, ?it/s]

process filters:  ASINH


 14%|█▍        | 1/7 [00:01<00:07,  1.31s/it]

process filters:  Linear


 29%|██▊       | 2/7 [00:02<00:06,  1.23s/it]

...

 71%|███████▏  | 5/7 [00:05<00:02,  1.17s/it]

process filters:  Sqrt


 86%|████████▌ | 6/7 [00:06<00:01,  1.15s/it]

process filters:  Squarded


100%|██████████| 7/7 [00:08<00:00,  1.17s/it]


# For testing set

In [71]:
# loop for saving all files as a new file
for index in tqdm.tqdm(range(len(sub_folder_test))):
    fil_name = sub_folder_test[index].split("\\")[-1].split(".")[0]
    print('process filters: ', fil_name)
    sub_iterate_labels = glob.glob(os.path.join(sub_folder_test[index], "*.txt"))
    sub_itearate_img = glob.glob(os.path.join(path_sub_img_test))[index]
    sub_itearate_img = glob.glob(os.path.join(sub_itearate_img, "*.jpg"))
    # print(len(sub_iterate_labels))
    # print(len(sub_itearate_img))

    for index_ in tqdm.tqdm(range(len(sub_itearate_img))):
        img_name = sub_itearate_img[index_].split("\\")[-1].split(".")[0]
        txt_name = sub_iterate_labels[index_].split("\\")[-1].split(".")[0]
        if img_name == txt_name:
            # read image with cv2
            img = cv2.imread(sub_itearate_img[index_], 0)
            with open(sub_iterate_labels[index_], "r") as f:
                items_list = f.readlines()
                items_list = [item.strip().split(" ") for item in items_list]
                # print("Processing file: ", txt_name)
                convert_position_to_YOLO_format(items_list, img, txt_name + ".txt",
                    path_save_test + fil_name + "/")
        else:
            print("File name is not match, Example: ", img_name, txt_name)

  0%|          | 0/7 [00:00<?, ?it/s]

process filters:  ASINH


 14%|█▍        | 1/7 [00:00<00:03,  1.82it/s]

process filters:  Linear


 29%|██▊       | 2/7 [00:00<00:02,  2.26it/s]

...

 71%|███████▏  | 5/7 [00:02<00:00,  2.42it/s]

process filters:  Sqrt


 86%|████████▌ | 6/7 [00:02<00:00,  2.49it/s]

process filters:  Squarded


100%|██████████| 7/7 [00:02<00:00,  2.42it/s]
